<a href="https://colab.research.google.com/github/danagtoma/ObjectDetection-AI/blob/main/aiProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training

In [ ]:
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
!pip install -qr requirements.txt 
import torch

from IPython.display import Image, clear_output  
from utils.downloads import attempt_download  

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

In [ ]:
%cd /content/yolov5

from roboflow import Roboflow
rf = Roboflow(api_key="9jbBnVfUV3TijnYb7hjK")
project = rf.workspace("learn-pkxkm").project("digit-bp4ww")
dataset = project.version(1).download("yolov5")

In [ ]:
%cat {dataset.location}/data.yaml

In [ ]:
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

nc: {num_classes} 
depth_multiple: 0.33 
width_multiple: 0.50  

anchors:
  - [10,13, 16,30, 33,23]  
  - [30,61, 62,45, 59,119]  
  - [116,90, 156,198, 373,326] 

backbone:
  [[-1, 1, Focus, [64, 3]],  
   [-1, 1, Conv, [128, 3, 2]],  
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]], 
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  
  ]

head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]], 
   [-1, 3, BottleneckCSP, [512, False]],  

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]], 
   [-1, 3, BottleneckCSP, [256, False]],  

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],   
   [-1, 3, BottleneckCSP, [512, False]],  

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  
   [-1, 3, BottleneckCSP, [1024, False]],  

   [[17, 20, 23], 1, Detect, [nc, anchors]],  
  ]

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 288 --batch 32 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# Evaluate Custom YOLOv5 Detector Performance

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results2/val_batch0_labels.jpg', width=900)

In [ ]:
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results2/train_batch0.jpg', width=900)

#Run Inference  With Trained Weights


In [ ]:
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results2/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Digit-1/test/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

# Export Trained Weights for Future Inference




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp /content/yolov5/runs/train/yolov5s_results2/weights/best.pt /content/gdrive/My\ Drive/AI/numbers

# Test On Other Images



In [ ]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

In [ ]:
!pip install -qr requirements.txt 

In [ ]:
!python detect.py --weights /content/yolov5/numbers2.pt --img 416 --conf 0.4 --source /content/yolov5/testnumbers

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/.jpg'):
    display(Image(filename=imageName))
    print("\n")

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): 
    display(Image(filename=imageName))
    print("\n")



In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp3/*.png'): 
    display(Image(filename=imageName))
    print("\n")